In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import json

In [2]:
def get_File(file_dir):
    images = []
    subfolders = []
    dict_A = {}

    for dirPath, dirNames, fileNames in os.walk(file_dir):
        for name in fileNames:
            images.append(os.path.join(dirPath, name))
            name = name.split("_")[0]
            dict_A[name+"_"] = 0

        for name in dirNames:
            subfolders.append(os.path.join(dirPath, name))

    # To record the labels of the image dataset
    labels = []
    count = 0
    for a_folder in subfolders:
        n_img = len(os.listdir(a_folder))
        labels = np.append(labels, n_img * [count])
        name = a_folder.split('\\')[-1]
        dict_A[name] = count
        
        count+=1

    subfolders = np.array([images, labels])
    subfolders = subfolders.transpose()

    image_list = list(subfolders[:, 0])
    label_list = list(subfolders[:, 1])
    label_list = [int(float(i)) for i in label_list]
    
    state = np.random.get_state()
    np.random.shuffle(image_list)
    np.random.set_state(state)
    np.random.shuffle(label_list)
    with open('CompareChart2.json', 'w') as f:
        json.dump(dict_A, f)
    
    return image_list, label_list

In [3]:
image_list, label_list = get_File("pic")
print(len(image_list))

1302


In [4]:
def int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# 轉Bytes資料為 tf.train.Feature 格式
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to_TFRecord(images, labels, filename):
    n_samples = len(labels)
    TFWriter = tf.python_io.TFRecordWriter(filename)

    print('\nTransform start...')
    for i in np.arange(0, n_samples):
        try:
            image = cv2.imread(images[i])
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            height,width,channel = image.shape
            image = cv2.resize(image,(224,224))
            
            if image is None:
                print('Error image:' + images[i])
            else:
                image_raw = image.tostring()

            label = int(labels[i])
            
            # 將 tf.train.Feature 合併成 tf.train.Features
            ftrs = tf.train.Features(
                    feature={'Label': int64_feature(label),
                             'image_raw': bytes_feature(image_raw),
                             'channel':int64_feature(channel),
                             'width':int64_feature(width),
                             'height':int64_feature(height)}
                   )
        
            # 將 tf.train.Features 轉成 tf.train.Example
            example = tf.train.Example(features=ftrs)

            # 將 tf.train.Example 寫成 tfRecord 格式
            TFWriter.write(example.SerializeToString())
        except IOError as e:
            print('Skip!\n')

    TFWriter.close()
    print('Transform done!')

In [5]:
convert_to_TFRecord(image_list, label_list, './data/Train2.tfrecords')


Transform start...
Transform done!
